# VQE of a spin-1/2 XXZ model with a particle-number conserving circuit ansatz 

In [ ]:
# necessary libraries and modules
from Ansatzes_Hamiltonians_class import AnsatzCircuit
from Ansatzes_Hamiltonians_class import XXZ
from qiskit.quantum_info import SparsePauliOp
import numpy as np
from qiskit.circuit import ParameterVector
from qiskit.primitives import StatevectorEstimator
from scipy.optimize import minimize 

In [ ]:
# simulation parameters
num_sites = 4
gate_class = "A"
entangle_initial_state = "no"
num_params_per_gate = 2
num_trials = 10  
num_opt_steps = 100
spin_model = "XXZ"   # list = ["XXZ", "XY"]
circ_type = "BWC"  # list = ["BWC", "LC"]; Brick-wall circuit (BWC) or linear circuit (LC)

In [ ]:
# define the quantum circuit
def build_circuit(num_sites, gate_class="A", entangle_initial_state="no", circ_type='BWC', num_params_per_gate=2):

    global num_params
    
    num_particles = num_sites // 2  # since we are using a number-conserved ansatz, XXZ corresponds to half-filling
     
    qc = AnsatzCircuit(
        num_particles,
        num_sites,
        num_params_per_gate,
        gate_class,
        entangle_initial_state, 
        circ_type  
    )
    
    if (qc.dim_subspace < 10):
        print("The quantum circuit: ")
        print(qc.draw())
    
    num_params = qc.num_parameters
    
    # decompose circuit before running
    qc_elem = qc.decompose().decompose() 

    return  qc_elem

In [ ]:
# Define the model Hamiltonian
def get_Hamiltonian(num_sites, spin_model):
    global ref_value 

    if spin_model == "XXZ":
        amp = 1
        H_obj = XXZ(num_sites, amp)                
    elif spin_model == "XY":        
        amp = 0
        H_obj = XXZ(num_sites, amp)   # XY = XXZ if ZZ term is turned off

    H = H_obj.Hamiltonian()
    H_op = SparsePauliOp.from_list(H)

    # get the reference ground state energy for small system sizes
    ref_value = H_obj.get_minimum_eigenvalue(H_op.to_matrix())
    print(f"\n Exact ground state energy: {ref_value}")
    
    return H_op

In [ ]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
            
    # create a list of pubs to evaluate, where each pub = (ansatz, hamiltonian, params)    
    pubs = []
    for j in range(len(hamiltonian)):
        hamiltonian_term = hamiltonian[j]
        qc_copy = ansatz.copy()
        pubs.append((qc_copy, hamiltonian_term, params))

    # run all the pubs job
    job = estimator.run(pubs)
    result = job.result()   # a blocking call, will wait until the job completes
    energy_list = [result[k].data.evs for k in range(len(result))]
    energy = np.sum(energy_list)

    #job_id = job.job_id()

    cost_history_dict["iters"] += 1
    cost_history_dict["param_vector"].append(params)
    cost_history_dict["cost_history"].append(energy)
    
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")

    return energy

In [ ]:
cost_history_dict = {
    "param_vector": [],
    "iters": -1,
    "cost_history": [],
    "std_history": [],
    "job_ids" : []
}

In [ ]:
# circuit and Hamiltonian objects
qc = build_circuit(num_sites, gate_class, entangle_initial_state, circ_type)
hamiltonian = get_Hamiltonian(num_sites, spin_model)

In [ ]:
estimator = StatevectorEstimator()    # for simulator

rng = np.random.default_rng(seed=71)
x0 = 2 * np.pi * rng.random(qc.num_parameters)

# save initial cost value
cost_func(x0, qc, hamiltonian, estimator)
# minimization of cost function
res = minimize(
    cost_func,
    x0,
    args=(qc, hamiltonian, estimator),
    method="cobyla",
    tol=10**-8,
    #options={"maxiter" : max_iter, "disp" : False}
)

In [ ]:
res

In [ ]:
print(f"\n Final estimated ground state energy: {res.fun}")
print(f" Relative error: {abs((res.fun - ref_value)/ref_value)}")